In [1]:
pip install SpeechRecognition pydub pyannote.audio

  Using cached pyannote.audio-3.3.2-py2.py3-none-any.whl.metadata (11 kB)
  Using cached asteroid_filterbanks-0.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached huggingface_hub-0.25.1-py3-none-any.whl.metadata (13 kB)
  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached pyannote.database-5.1.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached pyannote.metrics-3.2.1-py3-none-any.whl.metadata (1.3 kB)
  Using cached pyannote.pipeline-3.0.1-py3-none-any.whl.metadata (897 bytes)
  Using cached pytorch_metric_learning-2.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached speechbrain-1.0.1-py3-none-any.whl.metadata (24 kB)
  Using cached torch_audiomentations-0.11.1-py3-none-any.whl.metadata (14 kB)
  Using cached torchaudio-2.4.1-cp310-cp310-win_amd64.whl.metadata (6.5 kB)
  Using cached torchmetrics-1.4.2-py3-none-any.whl.metadata (19 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached typer-0.12.5-py3-none-any.whl.me


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Audio to text convertor python code:

In [3]:
import speech_recognition as sr
from pyannote.audio import Pipeline

# Function to convert speech to text using SpeechRecognition library
def convert_audio_to_text(audio_file):
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

    try:
        # Convert audio to text
        text = recognizer.recognize_google(audio)
        print("Audio to text conversion successful.")
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error requesting results from Google Speech Recognition service; {e}")
        return None

# Function to diarize the audio (identify speakers)
def diarize_audio(audio_file):
    # Load pre-trained diarization pipeline from pyannote
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

    # Perform diarization
    diarization = pipeline(audio_file)

    # Store speaker segments
    speaker_segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        speaker_segments.append((speaker, turn.start, turn.end))
    return speaker_segments

# Function to write text with speaker labels
def write_conversation_with_speakers(speaker_segments, text, output_file):
    with open(output_file, 'w') as file:
        current_speaker = None
        text_split = text.split('. ')  # Split the conversation roughly by sentences

        for i, sentence in enumerate(text_split):
            speaker = speaker_segments[i][0] if i < len(speaker_segments) else "Unknown"
            if speaker != current_speaker:
                current_speaker = speaker
                file.write(f'\n{current_speaker}: ')
            file.write(sentence + ". ")

# Main process to handle single or multi-speaker audio
def process_audio(audio_file, output_file):
    # Step 1: Convert audio to text
    text = convert_audio_to_text(audio_file)
    if not text:
        return

    # Step 2: Perform speaker diarization
    speaker_segments = diarize_audio(audio_file)

    # Step 3: Write the conversation to a .txt file
    write_conversation_with_speakers(speaker_segments, text, output_file)
    print(f"Conversation saved to {output_file}")

# Example usage
audio_file = r"C:\Users\omkar\Downloads\EnglishAudio.mp3"
output_file = r"C:\Users\omkar\Downloads\ConvertTxt.txt"
process_audio(audio_file, output_file)


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [4]:
pip install pydub


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pydub import AudioSegment
import speech_recognition as sr
from pyannote.audio import Pipeline
import os

# Function to convert MP3 to WAV using pydub
def convert_audio_format(input_file, output_format="wav"):
    try:
        audio = AudioSegment.from_file(input_file)
        output_file = input_file.rsplit('.', 1)[0] + f".{output_format}"
        audio.export(output_file, format=output_format)
        print(f"Converted {input_file} to {output_file}")
        return output_file
    except Exception as e:
        print(f"Error converting file: {e}")
        return None

# Function to convert speech to text using SpeechRecognition library
def convert_audio_to_text(audio_file):
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

    try:
        # Convert audio to text
        text = recognizer.recognize_google(audio)
        print("Audio to text conversion successful.")
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error requesting results from Google Speech Recognition service; {e}")
        return None

# Function to diarize the audio (identify speakers)
def diarize_audio(audio_file):
    # Load pre-trained diarization pipeline from pyannote
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

    # Perform diarization
    diarization = pipeline(audio_file)

    # Store speaker segments
    speaker_segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        speaker_segments.append((speaker, turn.start, turn.end))
    return speaker_segments

# Function to write text with speaker labels
def write_conversation_with_speakers(speaker_segments, text, output_file):
    with open(output_file, 'w') as file:
        current_speaker = None
        text_split = text.split('. ')  # Split the conversation roughly by sentences

        for i, sentence in enumerate(text_split):
            speaker = speaker_segments[i][0] if i < len(speaker_segments) else "Unknown"
            if speaker != current_speaker:
                current_speaker = speaker
                file.write(f'\n{current_speaker}: ')
            file.write(sentence + ". ")

# Main process to handle single or multi-speaker audio
def process_audio(audio_file, output_file):
    # Step 1: Convert to WAV if necessary
    if not audio_file.endswith(".wav"):
        audio_file = convert_audio_format(audio_file, "wav")
        if not audio_file:
            print("Failed to convert audio format.")
            return

    # Step 2: Convert audio to text
    text = convert_audio_to_text(audio_file)
    if not text:
        return

    # Step 3: Perform speaker diarization
    speaker_segments = diarize_audio(audio_file)

    # Step 4: Write the conversation to a .txt file
    write_conversation_with_speakers(speaker_segments, text, output_file)
    print(f"Conversation saved to {output_file}")

# Example usage
audio_file = r"C:\Users\omkar\Downloads\EnglishAudio.mp3"  # Replace with your actual file path
output_file = r"C:\Users\omkar\Downloads\ConvertTxt.txt"
process_audio(audio_file, output_file)


Error converting file: [WinError 2] The system cannot find the file specified
Failed to convert audio format.


In [4]:
ffmpeg -version


NameError: name 'ffmpeg' is not defined

In [6]:
from pydub import AudioSegment
import os

# Set ffmpeg path explicitly if necessary
AudioSegment.converter = r"C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"  # Make sure this is the correct path to your ffmpeg.exe

def convert_audio_format(input_file, output_format="wav"):
    try:
        # Convert the audio file
        audio = AudioSegment.from_file(input_file)
        output_file = input_file.rsplit('.', 1)[0] + f".{output_format}"
        audio.export(output_file, format=output_format)
        print(f"Converted {input_file} to {output_file}")
        return output_file
    except Exception as e:
        print(f"Error converting file: {e}")
        return None

# Example usage
audio_file = r"C:\Users\omkar\Downloads\EnglishAudio.mp3"
output_format = r"C:\Users\omkar\Downloads\EnglishAudio.txt"
convert_audio_format(audio_file, output_format)


Error converting file: [WinError 2] The system cannot find the file specified


In [7]:
from pydub import AudioSegment
import os

# Set ffmpeg path explicitly if necessary
AudioSegment.converter = r"C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"  # Ensure this is correct

def convert_audio_format(input_file, output_format="wav"):
    print(f"Input file: {input_file}")
    print(f"Output format: {output_format}")
    print(f"FFmpeg path: {AudioSegment.converter}")

    # Convert the audio file
    audio = AudioSegment.from_file(input_file)
    print("Audio file loaded successfully.")

    # Generate output filename
    output_file = input_file.rsplit('.', 1)[0] + f".{output_format}"
    print(f"Output file path: {output_file}")

    # Export the converted audio
    audio.export(output_file, format=output_format)
    print(f"Converted {input_file} to {output_file}")
    return output_file

# Example usage
audio_file = r"C:\Users\omkar\Downloads\audio.mp3"
output_format = "wav"
convert_audio_format(audio_file, output_format)


Input file: C:\Users\omkar\Downloads\audio.mp3
Output format: wav
FFmpeg path: C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [8]:
from pydub import AudioSegment
import os

# Set ffmpeg path explicitly if necessary
AudioSegment.converter = r"C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"  # Ensure this is correct

def convert_audio_format(input_file, output_file):
    print(f"Input file: {input_file}")
    print(f"Output file: {output_file}")
    print(f"FFmpeg path: {AudioSegment.converter}")

    # Convert the audio file
    audio = AudioSegment.from_file(input_file)
    print("Audio file loaded successfully.")

    # Extract the output format from the file extension
    output_format = output_file.rsplit('.', 1)[-1]
    print(f"Output format: {output_format}")

    # Export the converted audio
    audio.export(output_file, format=output_format)
    print(f"Converted {input_file} to {output_file}")
    return output_file

# Example usage
audio_file = r"C:\Users\omkar\Downloads\audio.mp3"
output_file = r"C:\Users\omkar\Downloads\ConvertedAudio.wav"  # Here you specify the full path to the output file
convert_audio_format(audio_file, output_file)


Input file: C:\Users\omkar\Downloads\audio.mp3
Output file: C:\Users\omkar\Downloads\ConvertedAudio.wav
FFmpeg path: C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [1]:
from pydub import AudioSegment
import os

# Set ffmpeg path explicitly if necessary
AudioSegment.converter = r"C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"

def convert_audio_format(input_file, output_format="wav"):
    # Check if the input file exists
    if not os.path.exists(input_file):
        print(f"Error: {input_file} not found.")
        return

    print(f"Input file: {input_file}")
    print(f"Output format: {output_format}")
    print(f"FFmpeg path: {AudioSegment.converter}")

    # Load the audio file
    audio = AudioSegment.from_file(input_file)
    print("Audio file loaded successfully.")

    # Generate output file path
    output_file = os.path.splitext(input_file)[0] + f".{output_format}"
    print(f"Output file path: {output_file}")

    # Export the converted audio
    audio.export(output_file, format=output_format)
    print(f"Converted {input_file} to {output_file}")
    return output_file

# Example usage
audio_file = r"C:\Users\omkar\Downloads\audio.mp3"
output_format = "wav"
convert_audio_format(audio_file, output_format)


C:\Users\omkar\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\omkar\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


Input file: C:\Users\omkar\Downloads\audio.mp3
Output format: wav
FFmpeg path: C:\ffmpeg\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe


FileNotFoundError: [WinError 2] The system cannot find the file specified